# 检索器模块 (Retriever Module)

该模块实现了基于向量相似性的问答检索系统，包含以下功能：
1. 文本向量化 - 使用sentence-transformers将中文问题转换为向量
2. 索引构建 - 使用FAISS构建高效的向量索引
3. ID映射 - 建立索引ID到原始数据的映射关系
4. 检索接口 - 提供search(query, k)函数返回最相似的问题


In [16]:
# 导入必要的库
import json
import numpy as np
import torch
import faiss
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
import os
import pickle

# 设置设备
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"使用设备: {device}")

# 设置文件路径
DATA_PATH = "../data/myoffer.json"
VECTORS_PATH = "../retriever_newdata/qa_tensors.pt"
INDEX_PATH = "../retriever_newdata/qa_faiss_index.index"
ID_MAP_PATH = "../retriever_newdata/id_map.json"


使用设备: cpu


In [19]:
# 1. 数据加载和预处理
def load_qa_data(data_path):
    """加载QA数据集"""
    with open(data_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    print(f"加载了 {len(data)} 条QA数据")
    return data

# 加载数据
qa_data = load_qa_data(DATA_PATH)

# 提取所有问题文本
questions = [item['title'] for item in qa_data]
print(f"提取了 {len(questions)} 个问题")
print("前5个问题示例:")
for i, q in enumerate(questions[:5]):
    print(f"{i+1}. {q}")


加载了 3406 条QA数据
提取了 3406 个问题
前5个问题示例:
1. 2025澳洲大学留学一年多少钱
2. 2025澳洲八大名校研究生申请条件有哪些
3. 墨尔本皇家理工大学申请条件有哪些
4. 澳洲纽卡斯尔大学申请条件有哪些
5. 麦考瑞大学申请条件有哪些


In [20]:
# 2. 文本向量化
def vectorize_questions(questions, model_name='paraphrase-multilingual-MiniLM-L12-v2'):
    """
    使用sentence-transformers将问题文本转换为向量
    选择支持中文的多语言模型
    """
    print(f"正在加载模型: {model_name}")
    model = SentenceTransformer(model_name, device=device)
    
    print("开始向量化问题...")
    # 批量编码，提高效率
    question_vectors = model.encode(
        questions,
        batch_size=32,
        show_progress_bar=True,
        convert_to_tensor=True,
        device=device
    )
    
    print(f"向量化完成！向量维度: {question_vectors.shape}")
    return question_vectors, model

# 执行向量化
question_vectors, encoder_model = vectorize_questions(questions)

# 保存向量张量
torch.save(question_vectors, VECTORS_PATH)
print(f"向量已保存到: {VECTORS_PATH}")


正在加载模型: paraphrase-multilingual-MiniLM-L12-v2
开始向量化问题...


Batches:   0%|          | 0/107 [00:00<?, ?it/s]

向量化完成！向量维度: torch.Size([3406, 384])
向量已保存到: ../retriever_newdata/qa_tensors.pt


In [21]:
# 3. 构建FAISS索引
def build_faiss_index(vectors):
    """构建FAISS索引以实现高效向量检索"""
    # 转换为numpy数组（FAISS需要）
    if torch.is_tensor(vectors):
        vectors_np = vectors.cpu().numpy().astype('float32')
    else:
        vectors_np = np.array(vectors, dtype='float32')
    
    # 获取向量维度
    dimension = vectors_np.shape[1]
    print(f"构建FAISS索引，向量维度: {dimension}")
    
    # 创建索引（使用L2距离的平面索引）
    index = faiss.IndexFlatL2(dimension)
    
    # 添加向量到索引
    index.add(vectors_np)
    
    print(f"索引构建完成！包含 {index.ntotal} 个向量")
    return index

# 构建索引
faiss_index = build_faiss_index(question_vectors)

# 保存索引
faiss.write_index(faiss_index, INDEX_PATH)
print(f"FAISS索引已保存到: {INDEX_PATH}")


构建FAISS索引，向量维度: 384
索引构建完成！包含 3406 个向量
FAISS索引已保存到: ../retriever_newdata/qa_faiss_index.index


In [26]:
# 4. 创建ID映射
def create_id_mapping(qa_data):
    """创建索引ID到原始数据的映射"""
    id_mapping = {}
    
    for idx, item in enumerate(qa_data):
        # 使用字符串作为键，与JSON格式保持一致
        id_mapping[str(idx)] = {
            'original_id': item['id'],
            'question': item['title'],
            'answer': item['raw_text'],
            'link': item['link'],
            'tags': item['tags']
        }
    
    print(f"创建了 {len(id_mapping)} 个ID映射")
    return id_mapping

# 创建映射
id_mapping = create_id_mapping(qa_data)

# 保存映射
with open(ID_MAP_PATH, 'w', encoding='utf-8') as f:
    json.dump(id_mapping, f, ensure_ascii=False, indent=2)

print(f"ID映射已保存到: {ID_MAP_PATH}")

# 显示映射示例
print("\\n映射示例 (前3个):")
for i in range(min(3, len(id_mapping))):
    print(f"索引 {i}: {id_mapping[str(i)]['question']}")


创建了 3406 个ID映射
ID映射已保存到: ../retriever_newdata/id_map.json
\n映射示例 (前3个):
索引 0: 2025澳洲大学留学一年多少钱
索引 1: 2025澳洲八大名校研究生申请条件有哪些
索引 2: 墨尔本皇家理工大学申请条件有哪些


## 检索接口 (Search API)

实现基于向量相似性的问题检索功能


In [27]:
class QARetriever:
    """问答检索器类"""
    
    def __init__(self, model_name='paraphrase-multilingual-MiniLM-L12-v2'):
        self.model_name = model_name
        self.encoder = None
        self.index = None
        self.id_mapping = None
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
    def load_model(self):
        """加载编码模型"""
        print(f"加载编码模型: {self.model_name}")
        self.encoder = SentenceTransformer(self.model_name, device=self.device)
        
    def load_index(self, index_path):
        """加载FAISS索引"""
        print(f"加载FAISS索引: {index_path}")
        self.index = faiss.read_index(index_path)
        
    def load_id_mapping(self, mapping_path):
        """加载ID映射"""
        print(f"加载ID映射: {mapping_path}")
        with open(mapping_path, 'r', encoding='utf-8') as f:
            self.id_mapping = json.load(f)
    
    def search(self, query, k=5):
        """
        检索最相似的K个问题
        
        Args:
            query (str): 用户查询问题
            k (int): 返回结果数量，默认5个
            
        Returns:
            list: 包含相似问题信息的列表
        """
        if not all([self.encoder, self.index, self.id_mapping]):
            raise ValueError("请先加载模型、索引和ID映射!")
        
        # 1. 将查询编码为向量
        query_vector = self.encoder.encode([query], convert_to_tensor=True, device=self.device)
        query_vector_np = query_vector.cpu().numpy().astype('float32')
        
        # 2. 使用FAISS搜索最相似的向量
        distances, indices = self.index.search(query_vector_np, k)
        
        # 3. 根据索引获取完整信息
        results = []
        for i, (distance, idx) in enumerate(zip(distances[0], indices[0])):
            if str(idx) in self.id_mapping:
                item = self.id_mapping[str(idx)]
                result = {
                    'rank': i + 1,
                    'similarity_score': float(1 / (1 + distance)),  # 转换为相似度分数
                    'distance': float(distance),
                    'original_id': item['original_id'],
                    'question': item['question'],
                    'answer': item['answer'],
                    'link': item['link'],
                    'tags': item['tags']
                }
                results.append(result)
        
        return results
    
    def initialize(self, index_path, mapping_path):
        """初始化检索器"""
        self.load_model()
        self.load_index(index_path)
        self.load_id_mapping(mapping_path)
        print("检索器初始化完成!")

# 创建检索器实例
retriever = QARetriever()
retriever.initialize(INDEX_PATH, ID_MAP_PATH)


加载编码模型: paraphrase-multilingual-MiniLM-L12-v2
加载FAISS索引: ../retriever_newdata/qa_faiss_index.index
加载ID映射: ../retriever_newdata/id_map.json
检索器初始化完成!


## 测试检索功能


In [28]:
# 测试检索功能
def test_search(retriever, test_queries):
    """测试检索功能"""
    for query in test_queries:
        print(f"\\n查询: '{query}'")
        print("-" * 50)
        
        results = retriever.search(query, k=3)
        
        for result in results:
            print(f"排名 {result['rank']}: 相似度 {result['similarity_score']:.4f}")
            print(f"问题: {result['question']}")
            print(f"答案: {result['answer']}")
            print(f"链接: {result['link']}")
            if result['tags']:
                print(f"标签: {', '.join(result['tags'])}")
            print()

# 定义测试查询
test_queries = [
    "如何申请签证",
    "签证需要准备什么材料",
    "签证申请流程",
    "签证费用",
    "签证有效期",
    "签证类型",
    "签证申请表",
]

# 执行测试
test_search(retriever, test_queries)


\n查询: '如何申请签证'
--------------------------------------------------
排名 1: 相似度 0.1132
问题: 如何留学申请？出国留学申请步骤盘点
答案: 近年来很多同学会有意愿出国留学，提升学历的同时开拓见闻，那么如何留学申请？出国留学申请有哪些步骤？ 留学申请主要分五个步骤，具体包括：确定留学国家、目标院校和专业；满足目标课程的申请要求；递交院校申请；递交签证申请；规划留学行程。 出国留学申请步骤盘点—— 1、确定留学国家、目标院校和专业：美国、英国、澳洲、加拿大、新西兰等都是近年来的留学热门国家，同学们在确定留学国家、院校和专业时，要综合考虑院校排名、专业实力、职业方向、移民动态、个人兴趣等因素。一般热门留学院校中，美国申请难度较大，英国、澳洲申请相对容易，但多为自费。留学专业中商科最受中国学生青睐，医科、牙科和兽医科学等学费较贵，教育专业的语言要求较高，理工科有专业背景限制，MBA等课程还有工作经验限制。同学们需要对这些情况有个大体的把握。 2、满足目标课程的申请要求：确定了院校和专业后，就要看自己是否达到申请要求了，通常学校官网上都有标出明确的录取要求。在读生们可要抓紧最后的时间好好学习，不断提高自己的竞争能力，已经毕业并就业的申请学生要准备好工作证明等，不要存在太长的空档期。 3、递交院校申请：英语能力好的同学可以DIY申请，觉得麻烦的同学也可以找中介机构帮忙。具体申请需要准备的材料包括毕业证、成绩单、英语成绩、CV、PS、工作经验证明（若有）等，另外个别专业课程会要求提供GRE/GMAT、作品集等辅助材料，具体材料细节可参照目标学校的官网申请要求。 4、递交签证申请：申请院校成功后就能拿到offer，按要求缴纳学费后换取CAS（英国）或COE（澳洲），然后开始准备签证材料，通常包括护照、照片、户口本、身份证、出生公证、毕业证、保险凭证、存款证明、体检证明等。递交签证申请后耐心等待，一般1-2个月后可以下签。 5、规划留学行程：拿到学生签证后大家就可以规划自己的留学出行了，可以预定住宿，具体是住在学校宿舍还是外面租房，可综合考量成本、舒适度和安全度，建议新生选择学校宿舍。此外还要买好机票、列好行李清单、明确出入境注意事项等。 以上就是如何留学申请的相关介绍，希望能给同学们提供参考，如果大家还有

## 便捷的检索函数


In [29]:
# 创建便捷的全局搜索函数
def search(query, k=5):
    """
    便捷的搜索函数
    
    Args:
        query (str): 用户查询问题
        k (int): 返回结果数量，默认5个
        
    Returns:
        list: 包含相似问题信息的列表
    """
    return retriever.search(query, k)

# 使用示例
print("=== 检索器模块已就绪 ===")
print("使用方法:")
print("results = search('你的问题', k=5)")
print("\\n快速测试:")
results = search("如何买公交卡", k=2)
for result in results:
    print(f"- {result['question']} (相似度: {result['similarity_score']:.3f})")


=== 检索器模块已就绪 ===
使用方法:
results = search('你的问题', k=5)
\n快速测试:
- 考雅思需要什么条件?怎样报名呢? (相似度: 0.049)
- 托福报考条件有哪些 (相似度: 0.047)
